# Outil d'Analyse de Sentiment des Médias Sociaux

Ce notebook est une adaptation pour Google Colab de l'outil d'analyse de sentiment des médias sociaux. Il permet d'analyser le sentiment des publications sur Twitter, Facebook et Google Reviews.

## Fonctionnalités

- Extraction de données depuis Twitter (via snscrape)
- Analyse de sentiment avec TextBlob et Transformers
- Extraction de mots-clés
- Visualisation des résultats
- Génération de rapports

## Comment utiliser ce notebook

1. Exécutez les cellules dans l'ordre
2. Configurez les paramètres d'analyse dans la section dédiée
3. Lancez l'analyse et explorez les résultats


## 1. Installation des dépendances

Commençons par installer les bibliothèques nécessaires.

In [ ]:
# Installation des packages requis
!pip install textblob snscrape pandas numpy matplotlib seaborn wordcloud transformers tqdm ipywidgets scikit-learn

## 2. Configuration et importation des bibliothèques

In [ ]:
# Importation des bibliothèques standard
import os
import re
import time
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, Any, List, Optional, Tuple, Callable

# Bibliothèques de traitement de données
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# Bibliothèques NLP
from textblob import TextBlob
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# Bibliothèques de visualisation
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.dates as mdates

# Widgets pour l'interface utilisateur
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

# Configuration de base
plt.style.use('seaborn-v0_8-whitegrid')
sns.set(style=&quot;whitegrid&quot;)

# Téléchargement des ressources NLTK nécessaires
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

## 3. Définition des classes de base et des utilitaires

In [ ]:
# Configuration du logging
def setup_logger():
    &quot;&quot;&quot;Configure le système de logging&quot;&quot;&quot;
    logger = logging.getLogger('SocialMediaAnalyzer')
    logger.setLevel(logging.INFO)
    
    # Handler pour afficher les logs dans le notebook
    handler = logging.StreamHandler()
    formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
    handler.setFormatter(formatter)
    
    logger.addHandler(handler)
    return logger

logger = setup_logger()

# Classe d'exception personnalisée
class ExtractionError(Exception):
    &quot;&quot;&quot;Exception levée lors d'erreurs d'extraction de données&quot;&quot;&quot;
    pass

class RateLimitError(ExtractionError):
    &quot;&quot;&quot;Exception levée lors de dépassement de limite de taux&quot;&quot;&quot;
    pass

class AuthenticationError(ExtractionError):
    &quot;&quot;&quot;Exception levée lors d'erreurs d'authentification&quot;&quot;&quot;
    pass

# Classe de validation des données
class DataValidator:
    &quot;&quot;&quot;Validateur de données pour l'analyse de sentiment&quot;&quot;&quot;
    
    def validate_text_content(self, text: str) -> bool:
        &quot;&quot;&quot;Valide le contenu textuel&quot;&quot;&quot;
        if not text or not isinstance(text, str):
            return False
        
        # Vérifier la longueur minimale (au moins 3 mots)
        words = text.split()
        if len(words) < 3:
            return False
        
        # Vérifier que le texte n'est pas juste des URLs ou des mentions
        cleaned_text = re.sub(r'https?://\S+|www\.\S+|@\w+', '', text).strip()
        if not cleaned_text or len(cleaned_text.split()) < 2:
            return False
        
        return True
    
    def validate_date_range(self, start_date: datetime, end_date: datetime) -> bool:
        &quot;&quot;&quot;Valide une plage de dates&quot;&quot;&quot;
        if not isinstance(start_date, datetime) or not isinstance(end_date, datetime):
            return False
        
        # La date de début doit être antérieure à la date de fin
        if start_date >= end_date:
            return False
        
        # La plage ne doit pas être trop grande (ex: max 1 an)
        if (end_date - start_date).days > 365:
            return False
        
        return True

## 4. Extracteur de données Twitter

In [ ]:
# Classe de base pour les extracteurs
class BaseExtractor:
    &quot;&quot;&quot;Classe de base pour les extracteurs de données&quot;&quot;&quot;
    
    def __init__(self, service: str, max_posts: int = 500):
        self.service = service
        self.max_posts = max_posts
        self.posts_extracted = 0
        self.errors_count = 0
        self.validator = DataValidator()
    
    def extract_posts(self, days: int = 30, **kwargs) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Méthode à implémenter dans les sous-classes&quot;&quot;&quot;
        raise NotImplementedError(&quot;Cette méthode doit être implémentée dans les sous-classes&quot;)
    
    def _calculate_date_range(self, days: int) -> Tuple[datetime, datetime]:
        &quot;&quot;&quot;Calcule la plage de dates pour l'extraction&quot;&quot;&quot;
        end_date = datetime.now()
        start_date = end_date - timedelta(days=days)
        return start_date, end_date
    
    def _validate_and_clean_post(self, post: Dict[str, Any]) -> Optional[Dict[str, Any]]:
        &quot;&quot;&quot;Valide et nettoie un post&quot;&quot;&quot;
        # Vérifier que le post contient du texte
        text = post.get('text', '')
        if not self.validator.validate_text_content(text):
            return None
        
        # Nettoyer le texte des URLs et autres éléments non pertinents
        post['text'] = re.sub(r'https?://\S+|www\.\S+', '', text)
        
        return post
    
    def _normalize_post_structure(self, post: Dict[str, Any], source: str) -> Dict[str, Any]:
        &quot;&quot;&quot;Normalise la structure d'un post pour un traitement uniforme&quot;&quot;&quot;
        normalized = {
            'id': post.get('id', str(hash(str(post)))),
            'text': post.get('text', ''),
            'created_at': post.get('created_at', datetime.now().isoformat()),
            'source': source,
            'service': self.service
        }
        
        # Ajouter des métriques si disponibles
        if 'likes' in post:
            normalized['likes'] = post['likes']
        if 'retweets' in post:
            normalized['shares'] = post['retweets']
        
        return normalized
    
    def _handle_extraction_error(self, error: Exception, context: str):
        &quot;&quot;&quot;Gère les erreurs d'extraction&quot;&quot;&quot;
        self.errors_count += 1
        
        if isinstance(error, RateLimitError):
            logger.warning(f&quot;Limite de taux atteinte pendant {context}: {error}&quot;)
            raise
        elif isinstance(error, AuthenticationError):
            logger.error(f&quot;Erreur d'authentification pendant {context}: {error}&quot;)
            raise
        else:
            logger.error(f&quot;Erreur pendant {context}: {error}&quot;)
            raise ExtractionError(f&quot;Erreur d'extraction: {error}&quot;)
    
    def _rate_limit_delay(self, seconds: float = 1.0):
        &quot;&quot;&quot;Ajoute un délai pour respecter les limites de taux&quot;&quot;&quot;
        time.sleep(seconds)
    
    def get_extraction_stats(self) -> Dict[str, Any]:
        &quot;&quot;&quot;Retourne les statistiques d'extraction&quot;&quot;&quot;
        return {
            'service': self.service,
            'posts_extracted': self.posts_extracted,
            'errors_count': self.errors_count,
            'success_rate': (self.posts_extracted / (self.posts_extracted + self.errors_count) * 100) 
                if (self.posts_extracted + self.errors_count) > 0 else 0
        }

In [ ]:
# Extracteur Twitter
class TwitterExtractor(BaseExtractor):
    &quot;&quot;&quot;Extracteur de données Twitter utilisant snscrape&quot;&quot;&quot;
    
    def __init__(self, service: str, max_posts: int = 500):
        super().__init__(service, max_posts)
        
        # Vérifier si snscrape est disponible
        try:
            import snscrape.modules.twitter as sntwitter
            self.snscrape_available = True
        except ImportError:
            self.snscrape_available = False
            logger.warning(&quot;snscrape n'est pas disponible. L'extraction Twitter sera limitée.&quot;)
    
    def extract_posts(self, days: int = 30, **kwargs) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les tweets avec snscrape&quot;&quot;&quot;
        logger.info(f&quot;Extraction des tweets pour '{self.service}' des {days} derniers jours&quot;)
        
        try:
            if self.snscrape_available:
                return self._extract_with_snscrape(days, **kwargs)
            else:
                raise ExtractionError(&quot;Aucune méthode d'extraction Twitter disponible&quot;)
                
        except Exception as e:
            self._handle_extraction_error(e, &quot;Extraction Twitter&quot;)
            return []
    
    def _extract_with_snscrape(self, days: int, **kwargs) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les tweets en utilisant snscrape&quot;&quot;&quot;
        import snscrape.modules.twitter as sntwitter
        
        posts = []
        start_date, end_date = self._calculate_date_range(days)
        
        try:
            query = self._build_search_query()
            
            # Construire la requête snscrape avec la plage de dates
            since_date = start_date.strftime(&quot;%Y-%m-%d&quot;)
            until_date = end_date.strftime(&quot;%Y-%m-%d&quot;)
            snscrape_query = f&quot;{query} since:{since_date} until:{until_date}&quot;
            
            logger.info(f&quot;Utilisation de snscrape avec la requête: {snscrape_query}&quot;)
            
            # Recherche de tweets avec barre de progression
            for i, tweet in enumerate(tqdm(
                sntwitter.TwitterSearchScraper(snscrape_query).get_items(),
                desc=&quot;Extraction des tweets&quot;,
                total=self.max_posts
            )):
                if len(posts) >= self.max_posts:
                    break
                
                processed_tweet = self._process_snscrape_tweet(tweet)
                if processed_tweet:
                    posts.append(processed_tweet)
                    self.posts_extracted += 1
                
                # Suivi de la progression
                if i % 50 == 0 and i > 0:
                    logger.info(f&quot;Traité {i} tweets, extrait {len(posts)}&quot;)
            
        except Exception as e:
            logger.error(f&quot;Erreur snscrape: {e}&quot;)
            raise ExtractionError(f&quot;L'extraction snscrape a échoué: {e}&quot;)
        
        logger.info(f&quot;Extrait {len(posts)} tweets via snscrape&quot;)
        return posts
    
    def _build_search_query(self) -> str:
        &quot;&quot;&quot;Construit la requête de recherche Twitter&quot;&quot;&quot;
        service_terms = self.service.lower().split()
        
        # Créer une requête avec le nom du service et ses variations
        query_parts = []
        for term in service_terms:
            query_parts.append(term)
            query_parts.append(f&quot;#{term}&quot;)
            query_parts.append(f&quot;@{term}&quot;)
        
        # Joindre les parties avec OR
        query = &quot; OR &quot;.join(query_parts)
        
        # Exclure les retweets pour du contenu original
        query += &quot; -is:retweet&quot;
        
        return query
    
    def _process_snscrape_tweet(self, tweet) -> Optional[Dict[str, Any]]:
        &quot;&quot;&quot;Traite un objet tweet snscrape&quot;&quot;&quot;
        try:
            tweet_data = {
                'id': str(tweet.id),
                'text': tweet.content,
                'created_at': tweet.date.isoformat(),
                'username': tweet.user.username,
                'display_name': tweet.user.displayname,
                'likes': tweet.likeCount,
                'retweets': tweet.retweetCount,
                'replies': tweet.replyCount,
                'quotes': tweet.quoteCount,
                'lang': getattr(tweet, 'lang', 'unknown')
            }
            
            # Valider et nettoyer
            validated_tweet = self._validate_and_clean_post(tweet_data)
            if validated_tweet:
                return self._normalize_post_structure(validated_tweet, 'twitter')
            
        except Exception as e:
            logger.error(f&quot;Erreur de traitement du tweet snscrape: {e}&quot;)
            self.errors_count += 1
        
        return None

## 5. Prétraitement de texte et analyse de sentiment

In [ ]:
# Préprocesseur de texte
class TextPreprocessor:
    &quot;&quot;&quot;Prétraitement de texte pour l'analyse de sentiment&quot;&quot;&quot;
    
    def __init__(self):
        self.stopwords_en = set(stopwords.words('english'))
        self.stopwords_fr = set(stopwords.words('french'))
    
    def preprocess_text(self, text: str, language: str = 'auto') -> Dict[str, Any]:
        &quot;&quot;&quot;Prétraite un texte pour l'analyse&quot;&quot;&quot;
        if not text:
            return {'cleaned': '', 'tokens': [], 'language': 'unknown', 'preprocessing_steps': []}
        
        preprocessing_steps = []
        
        # Détection de la langue si nécessaire
        if language == 'auto':
            language = self._detect_language(text)
        
        # Conversion en minuscules
        text = text.lower()
        preprocessing_steps.append('lowercase')
        
        # Suppression des URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text)
        preprocessing_steps.append('remove_urls')
        
        # Suppression des mentions (@user)
        text = re.sub(r'@\w+', '', text)
        preprocessing_steps.append('remove_mentions')
        
        # Suppression des hashtags (#topic)
        text = re.sub(r'#\w+', '', text)
        preprocessing_steps.append('remove_hashtags')
        
        # Suppression des caractères spéciaux et chiffres
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'\d+', '', text)
        preprocessing_steps.append('remove_special_chars')
        
        # Tokenisation
        tokens = word_tokenize(text)
        preprocessing_steps.append('tokenize')
        
        # Suppression des stop words
        if language == 'french':
            tokens = [word for word in tokens if word not in self.stopwords_fr]
        else:  # default to English
            tokens = [word for word in tokens if word not in self.stopwords_en]
        preprocessing_steps.append('remove_stopwords')
        
        # Reconstruction du texte nettoyé
        cleaned_text = ' '.join(tokens)
        
        return {
            'cleaned': cleaned_text,
            'tokens': tokens,
            'language': language,
            'preprocessing_steps': preprocessing_steps
        }
    
    def _detect_language(self, text: str) -> str:
        &quot;&quot;&quot;Détection simple de la langue pour l'analyse de sentiment&quot;&quot;&quot;
        # Heuristique simple basée sur les mots courants
        french_words = ['le', 'la', 'les', 'un', 'une', 'de', 'du', 'des', 'et', 'est', 'sont']
        english_words = ['the', 'and', 'is', 'are', 'in', 'on', 'at', 'to', 'for', 'of']
        
        text_lower = text.lower()
        
        french_score = sum(1 for word in french_words if f&quot; {word} &quot; in f&quot; {text_lower} &quot;)
        english_score = sum(1 for word in english_words if f&quot; {word} &quot; in f&quot; {text_lower} &quot;)
        
        if french_score > english_score:
            return 'french'
        else:
            return 'english'

In [ ]:
# Analyseur de sentiment
class SentimentAnalyzer:
    &quot;&quot;&quot;Analyseur de sentiment multi-modèles&quot;&quot;&quot;
    
    def __init__(self, model_type: str = 'auto', language: str = 'auto'):
        self.model_type = model_type
        self.language = language
        self.models = {}
        self._setup_models()
    
    def _setup_models(self):
        &quot;&quot;&quot;Configure les modèles d'analyse de sentiment&quot;&quot;&quot;
        try:
            # Vérifier si transformers est disponible
            try:
                from transformers import pipeline
                # Charger le modèle transformers pour l'analyse de sentiment
                self.models['transformers'] = pipeline(
                    &quot;sentiment-analysis&quot;,
                    model=&quot;cardiffnlp/twitter-roberta-base-sentiment-latest&quot;,
                    tokenizer=&quot;cardiffnlp/twitter-roberta-base-sentiment-latest&quot;
                )
                logger.info(&quot;Modèle Transformers chargé pour l'analyse de sentiment&quot;)
            except ImportError:
                logger.warning(&quot;La bibliothèque transformers n'est pas disponible&quot;)
            
            logger.info(&quot;Analyse de sentiment TextBlob disponible&quot;)
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la configuration des modèles de sentiment: {e}&quot;)
    
    def analyze_sentiment(self, text: str, language: Optional[str] = None) -> Dict[str, Any]:
        &quot;&quot;&quot;Analyse le sentiment d'un texte&quot;&quot;&quot;
        if not text or not isinstance(text, str):
            return self._get_neutral_result()
        
        try:
            lang = language or self.language or self._detect_language(text)
            
            # Choisir la méthode d'analyse
            if self.model_type == 'transformers' and 'transformers' in self.models:
                result = self._analyze_with_transformers(text)
            elif lang == 'french':
                result = self._analyze_with_textblob_fr(text)
            else:
                result = self._analyze_with_textblob_en(text)
            
            # Ajouter des métadonnées
            result.update({
                'text': text[:100] + '..' if len(text) > 100 else text,
                'language': lang,
                'model_used': self.model_type
            })
            
            return result
            
        except Exception as e:
            logger.error(f&quot;Erreur d'analyse de sentiment: {e}&quot;)
            return self._get_neutral_result()
    
    def analyze_batch(self, texts: List[str], language: Optional[str] = None) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Analyse le sentiment pour plusieurs textes&quot;&quot;&quot;
        results = []
        
        for text in tqdm(texts, desc=&quot;Analyse de sentiment&quot;):
            result = self.analyze_sentiment(text, language)
            results.append(result)
        
        return results
    
    def _analyze_with_transformers(self, text: str) -> Dict[str, Any]:
        &quot;&quot;&quot;Analyse le sentiment avec Transformers&quot;&quot;&quot;
        try:
            # Tronquer le texte si trop long
            max_length = 512
            if len(text) > max_length:
                text = text[:max_length]
            
            result = self.models['transformers'](text)[0]
            
            label = result['label'].lower()
            score = result['score']
            
            # Mapper au sentiment standard
            if 'positive' in label:
                sentiment = 'positive'
                polarity = score
            elif 'negative' in label:
                sentiment = 'negative'
                polarity = -score
            else:
                sentiment = 'neutral'
                polarity = 0
            
            return {
                'sentiment': sentiment,
                'polarity': polarity,
                'confidence': score,
                'raw_label': result['label'],
                'method': 'transformers'
            }
            
        except Exception as e:
            logger.error(f&quot;Erreur d'analyse Transformers: {e}&quot;)
            return self._analyze_with_textblob_en(text)  # Fallback
    
    def _analyze_with_textblob_en(self, text: str) -> Dict[str, Any]:
        &quot;&quot;&quot;Analyse le sentiment avec TextBlob (Anglais)&quot;&quot;&quot;
        try:
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            
            # Classifier le sentiment
            if polarity > 0.1:
                sentiment = 'positive'
            elif polarity < -0.1:
                sentiment = 'negative'
            else:
                sentiment = 'neutral'
            
            return {
                'sentiment': sentiment,
                'polarity': polarity,
                'subjectivity': subjectivity,
                'confidence': abs(polarity),
                'method': 'textblob_en'
            }
            
        except Exception as e:
            logger.error(f&quot;Erreur d'analyse TextBlob Anglais: {e}&quot;)
            return self._get_neutral_result()
    
    def _analyze_with_textblob_fr(self, text: str) -> Dict[str, Any]:
        &quot;&quot;&quot;Analyse le sentiment avec TextBlob (Français)&quot;&quot;&quot;
        try:
            # TextBlob fonctionne avec le texte français, bien que la précision puisse varier
            blob = TextBlob(text)
            polarity = blob.sentiment.polarity
            subjectivity = blob.sentiment.subjectivity
            
            # Ajuster les seuils pour le français (plus conservateur)
            if polarity > 0.2:
                sentiment = 'positive'
                confidence = polarity
            elif polarity < -0.2:
                sentiment = 'negative'
                confidence = abs(polarity)
            else:
                sentiment = 'neutral'
                confidence = 1.0 - abs(polarity)  # Confiance plus élevée pour neutre
            
            return {
                'sentiment': sentiment,
                'polarity': polarity,
                'subjectivity': subjectivity,
                'confidence': confidence,
                'method': 'textblob_fr'
            }
            
        except Exception as e:
            logger.error(f&quot;Erreur d'analyse TextBlob Français: {e}&quot;)
            return self._analyze_with_textblob_en(text)  # Fallback vers l'anglais
    
    def _detect_language(self, text: str) -> str:
        &quot;&quot;&quot;Détection simple de la langue pour l'analyse de sentiment&quot;&quot;&quot;
        # Heuristique simple basée sur les mots courants
        french_words = ['le', 'la', 'les', 'un', 'une', 'de', 'du', 'des', 'et', 'est', 'sont']
        english_words = ['the', 'and', 'is', 'are', 'in', 'on', 'at', 'to', 'for', 'of']
        
        text_lower = text.lower()
        
        french_score = sum(1 for word in french_words if f&quot; {word} &quot; in f&quot; {text_lower} &quot;)
        english_score = sum(1 for word in english_words if f&quot; {word} &quot; in f&quot; {text_lower} &quot;)
        
        if french_score > english_score:
            return 'french'
        else:
            return 'english'
    
    def _get_neutral_result(self) -> Dict[str, Any]:
        &quot;&quot;&quot;Retourne un résultat de sentiment neutre&quot;&quot;&quot;
        return {
            'sentiment': 'neutral',
            'polarity': 0.0,
            'subjectivity': 0.0,
            'confidence': 0.0,
            'method': 'fallback',
            'error': 'Analyse échouée'
        }
    
    def get_sentiment_summary(self, results: List[Dict[str, Any]]) -> Dict[str, Any]:
        &quot;&quot;&quot;Obtient des statistiques de résumé pour les résultats d'analyse de sentiment&quot;&quot;&quot;
        if not results:
            return {
                'total': 0,
                'positive': 0,
                'negative': 0,
                'neutral': 0,
                'percentages': {
                    'positive': 0,
                    'negative': 0,
                    'neutral': 0
                },
                'average_polarity': 0.0,
                'average_confidence': 0.0
            }
        
        total = len(results)
        positive = sum(1 for r in results if r['sentiment'] == 'positive')
        negative = sum(1 for r in results if r['sentiment'] == 'negative')
        neutral = sum(1 for r in results if r['sentiment'] == 'neutral')
        
        avg_polarity = np.mean([r['polarity'] for r in results])
        avg_confidence = np.mean([r.get('confidence', 0) for r in results])
        
        return {
            'total': total,
            'positive': positive,
            'negative': negative,
            'neutral': neutral,
            'percentages': {
                'positive': round(positive / total * 100, 2),
                'negative': round(negative / total * 100, 2),
                'neutral': round(neutral / total * 100, 2)
            },
            'average_polarity': round(avg_polarity, 3),
            'average_confidence': round(avg_confidence, 3)
        }

## 6. Extraction de mots-clés

In [ ]:
# Extracteur de mots-clés
class KeywordExtractor:
    &quot;&quot;&quot;Extracteur de mots-clés utilisant plusieurs méthodes&quot;&quot;&quot;
    
    def __init__(self, language: str = 'auto', max_keywords: int = 50):
        self.language = language
        self.max_keywords = max_keywords
        self.stopwords_en = set(stopwords.words('english'))
        self.stopwords_fr = set(stopwords.words('french'))
    
    def extract_keywords(self, texts: List[str], method: str = 'combined') -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les mots-clés des textes&quot;&quot;&quot;
        if not texts:
            return []
        
        # Déterminer la méthode d'extraction
        if method == 'tfidf':
            return self._extract_with_tfidf(texts)
        elif method == 'frequency':
            return self._extract_with_frequency(texts)
        else:  # combined
            tfidf_keywords = self._extract_with_tfidf(texts)
            freq_keywords = self._extract_with_frequency(texts)
            return self._combine_keyword_results(tfidf_keywords, freq_keywords)
    
    def _extract_with_tfidf(self, texts: List[str]) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les mots-clés en utilisant TF-IDF&quot;&quot;&quot;
        try:
            # Déterminer la langue pour les stop words
            if self.language == 'auto':
                # Utiliser l'anglais par défaut pour TF-IDF
                stop_words = self.stopwords_en
            elif self.language == 'french':
                stop_words = self.stopwords_fr
            else:
                stop_words = self.stopwords_en
            
            # Configurer le vectoriseur TF-IDF
            vectorizer = TfidfVectorizer(
                max_features=100,
                stop_words=list(stop_words),
                ngram_range=(1, 2),  # Unigrammes et bigrammes
                min_df=2  # Ignorer les termes qui apparaissent dans moins de 2 documents
            )
            
            # Calculer les scores TF-IDF
            tfidf_matrix = vectorizer.fit_transform(texts)
            feature_names = vectorizer.get_feature_names_out()
            
            # Calculer les scores moyens pour chaque terme
            tfidf_scores = np.array(tfidf_matrix.mean(axis=0)).flatten()
            
            # Créer un dictionnaire de scores
            keyword_scores = {feature_names[i]: tfidf_scores[i] for i in range(len(feature_names))}
            
            # Trier par score et convertir en liste de dictionnaires
            sorted_keywords = sorted(keyword_scores.items(), key=lambda x: x[1], reverse=True)
            
            # Limiter au nombre maximum de mots-clés
            top_keywords = sorted_keywords[:self.max_keywords]
            
            # Convertir en format standard
            result = [
                {
                    'keyword': kw,
                    'score': float(score),
                    'method': 'tfidf',
                    'frequency': self._count_term_frequency(kw, texts)
                }
                for kw, score in top_keywords
            ]
            
            return result
            
        except Exception as e:
            logger.error(f&quot;Erreur d'extraction TF-IDF: {e}&quot;)
            return []
    
    def _extract_with_frequency(self, texts: List[str]) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les mots-clés en utilisant la fréquence des termes&quot;&quot;&quot;
        try:
            # Déterminer la langue pour les stop words
            if self.language == 'auto':
                # Utiliser l'anglais par défaut
                stop_words = self.stopwords_en
            elif self.language == 'french':
                stop_words = self.stopwords_fr
            else:
                stop_words = self.stopwords_en
            
            # Configurer le vectoriseur de comptage
            vectorizer = CountVectorizer(
                max_features=100,
                stop_words=list(stop_words),
                ngram_range=(1, 2)  # Unigrammes et bigrammes
            )
            
            # Calculer les fréquences
            count_matrix = vectorizer.fit_transform(texts)
            feature_names = vectorizer.get_feature_names_out()
            
            # Calculer les fréquences totales
            count_scores = np.array(count_matrix.sum(axis=0)).flatten()
            
            # Créer un dictionnaire de fréquences
            keyword_counts = {feature_names[i]: int(count_scores[i]) for i in range(len(feature_names))}
            
            # Trier par fréquence et convertir en liste de dictionnaires
            sorted_keywords = sorted(keyword_counts.items(), key=lambda x: x[1], reverse=True)
            
            # Limiter au nombre maximum de mots-clés
            top_keywords = sorted_keywords[:self.max_keywords]
            
            # Normaliser les scores
            max_count = max([count for _, count in top_keywords]) if top_keywords else 1
            
            # Convertir en format standard
            result = [
                {
                    'keyword': kw,
                    'score': count / max_count,  # Score normalisé
                    'method': 'frequency',
                    'frequency': count
                }
                for kw, count in top_keywords
            ]
            
            return result
            
        except Exception as e:
            logger.error(f&quot;Erreur d'extraction par fréquence: {e}&quot;)
            return []
    
    def extract_key_phrases(self, texts: List[str]) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les phrases clés des textes&quot;&quot;&quot;
        # Méthode simplifiée pour extraire des phrases clés
        # Dans une implémentation complète, on pourrait utiliser des méthodes plus avancées
        try:
            # Combiner tous les textes
            combined_text = ' '.join(texts)
            
            # Extraire des phrases (séquences de 3-5 mots)
            words = combined_text.split()
            phrases = []
            
            for i in range(len(words) - 3):
                phrase = ' '.join(words[i:i+3])  # Phrases de 3 mots
                if len(phrase) > 10:  # Ignorer les phrases trop courtes
                    phrases.append(phrase)
            
            # Compter les occurrences
            phrase_counts = {}
            for phrase in phrases:
                phrase_counts[phrase] = phrase_counts.get(phrase, 0) + 1
            
            # Trier par fréquence
            sorted_phrases = sorted(phrase_counts.items(), key=lambda x: x[1], reverse=True)
            
            # Limiter et normaliser
            top_phrases = sorted_phrases[:20]  # Limiter à 20 phrases
            max_count = max([count for _, count in top_phrases]) if top_phrases else 1
            
            # Convertir en format standard
            result = [
                {
                    'keyword': phrase,
                    'score': count / max_count,  # Score normalisé
                    'method': 'phrase',
                    'frequency': count
                }
                for phrase, count in top_phrases
            ]
            
            return result
            
        except Exception as e:
            logger.error(f&quot;Erreur d'extraction de phrases clés: {e}&quot;)
            return []
    
    def _combine_keyword_results(self, tfidf_keywords: List[Dict[str, Any]], 
                               freq_keywords: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Combine les résultats de différentes méthodes d'extraction&quot;&quot;&quot;
        # Créer un dictionnaire pour fusionner les résultats
        combined = {}
        
        # Ajouter les mots-clés TF-IDF
        for kw in tfidf_keywords:
            combined[kw['keyword']] = {
                'keyword': kw['keyword'],
                'tfidf_score': kw['score'],
                'frequency': kw['frequency'],
                'methods': ['tfidf']
            }
        
        # Ajouter ou mettre à jour avec les mots-clés de fréquence
        for kw in freq_keywords:
            if kw['keyword'] in combined:
                combined[kw['keyword']]['freq_score'] = kw['score']
                combined[kw['keyword']]['methods'].append('frequency')
            else:
                combined[kw['keyword']] = {
                    'keyword': kw['keyword'],
                    'freq_score': kw['score'],
                    'frequency': kw['frequency'],
                    'methods': ['frequency']
                }
        
        # Calculer un score combiné
        for keyword, data in combined.items():
            tfidf_score = data.get('tfidf_score', 0)
            freq_score = data.get('freq_score', 0)
            
            # Score combiné: moyenne pondérée (TF-IDF a plus de poids)
            if 'tfidf_score' in data and 'freq_score' in data:
                combined_score = (tfidf_score * 0.7) + (freq_score * 0.3)
            elif 'tfidf_score' in data:
                combined_score = tfidf_score
            else:
                combined_score = freq_score
            
            data['score'] = combined_score
        
        # Convertir en liste et trier par score combiné
        result = list(combined.values())
        result.sort(key=lambda x: x['score'], reverse=True)
        
        # Limiter au nombre maximum de mots-clés
        return result[:self.max_keywords]
    
    def _count_term_frequency(self, term: str, texts: List[str]) -> int:
        &quot;&quot;&quot;Compte la fréquence d'un terme dans les textes&quot;&quot;&quot;
        count = 0
        for text in texts:
            count += text.lower().count(term.lower())
        return count

## 7. Visualisation des résultats

In [ ]:
# Générateur de graphiques
class ChartsGenerator:
    &quot;&quot;&quot;Génère divers graphiques pour l'analyse de sentiment&quot;&quot;&quot;
    
    def __init__(self, style: str = None):
        self.style = style or 'seaborn-v0_8-whitegrid'
        plt.style.use(self.style)
        self.colors = sns.color_palette(&quot;Set2&quot;)
    
    def create_sentiment_pie_chart(self, sentiment_summary: Dict[str, Any], 
                                title: str = &quot;Distribution des sentiments&quot;) -> plt.Figure:
        &quot;&quot;&quot;Crée un graphique circulaire pour la distribution des sentiments&quot;&quot;&quot;
        try:
            fig, ax = plt.subplots(figsize=(10, 8))
            
            # Préparer les données
            labels = ['Positif', 'Négatif', 'Neutre']
            sizes = [
                sentiment_summary['percentages']['positive'],
                sentiment_summary['percentages']['negative'],
                sentiment_summary['percentages']['neutral']
            ]
            colors = ['#2E8B57', '#DC143C', '#808080']  # Vert, Rouge, Gris
            
            # Créer le graphique circulaire
            wedges, texts, autotexts = ax.pie(
                sizes,
                labels=labels,
                colors=colors,
                autopct='%1.1f%%',
                startangle=90,
                explode=(0.05, 0.05, 0.05)
            )
            
            # Personnaliser
            ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
            
            # Style du texte
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
                autotext.set_fontsize(12)
            
            # Ajouter des informations de comptage
            total = sentiment_summary['total']
            counts = [
                sentiment_summary['positive'],
                sentiment_summary['negative'],
                sentiment_summary['neutral']
            ]
            
            legend_labels = [
                f&quot;{label}: {count} ({size:.1f}%)&quot; 
                for label, count, size in zip(labels, counts, sizes)
            ]
            
            ax.legend(
                wedges, legend_labels,
                title=&quot;Comptage des sentiments&quot;,
                loc=&quot;center left&quot;,
                bbox_to_anchor=(1, 0, 0.5, 1)
            )
            
            plt.tight_layout()
            
            return fig
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la création du graphique circulaire des sentiments: {e}&quot;)
            return self._create_error_chart(&quot;Graphique circulaire des sentiments&quot;)
    
    def create_sentiment_bar_chart(self, sentiment_summary: Dict[str, Any],
                                title: str = &quot;Résultats de l'analyse de sentiment&quot;) -> plt.Figure:
        &quot;&quot;&quot;Crée un graphique à barres pour l'analyse de sentiment&quot;&quot;&quot;
        try:
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
            
            # Graphique de gauche: Comptages
            sentiments = ['Positif', 'Négatif', 'Neutre']
            counts = [
                sentiment_summary['positive'],
                sentiment_summary['negative'],
                sentiment_summary['neutral']
            ]
            colors = ['#2E8B57', '#DC143C', '#808080']
            
            bars1 = ax1.bar(sentiments, counts, color=colors, alpha=0.8)
            ax1.set_title('Comptage des sentiments', fontsize=14, fontweight='bold')
            ax1.set_ylabel('Nombre de publications')
            
            # Ajouter des étiquettes de valeur sur les barres
            for bar, count in zip(bars1, counts):
                height = bar.get_height()
                ax1.text(
                    bar.get_x() + bar.get_width()/2., height + max(counts)*0.01,
                    f'{count}', ha='center', va='bottom', fontweight='bold'
                )
            
            # Graphique de droite: Pourcentages
            percentages = [
                sentiment_summary['percentages']['positive'],
                sentiment_summary['percentages']['negative'],
                sentiment_summary['percentages']['neutral']
            ]
            
            bars2 = ax2.bar(sentiments, percentages, color=colors, alpha=0.8)
            ax2.set_title('Pourcentages des sentiments', fontsize=14, fontweight='bold')
            ax2.set_ylabel('Pourcentage (%)')
            
            # Ajouter des étiquettes de pourcentage
            for bar, pct in zip(bars2, percentages):
                height = bar.get_height()
                ax2.text(
                    bar.get_x() + bar.get_width()/2., height + max(percentages)*0.01,
                    f'{pct:.1f}%', ha='center', va='bottom', fontweight='bold'
                )
            
            # Titre général
            fig.suptitle(title, fontsize=16, fontweight='bold', y=1.02)
            
            # Ajouter des statistiques
            stats_text = (
                f&quot;Total des publications: {sentiment_summary['total']}\n&quot;
                f&quot;Polarité moyenne: {sentiment_summary.get('average_polarity', 0):.3f}\n&quot;
                f&quot;Confiance moyenne: {sentiment_summary.get('average_confidence', 0):.3f}&quot;
            )
            
            fig.text(0.02, 0.02, stats_text, fontsize=10, 
                    bbox=dict(boxstyle=&quot;round,pad=0.3&quot;, facecolor=&quot;lightgray&quot;, alpha=0.5))
            
            plt.tight_layout()
            
            return fig
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la création du graphique à barres des sentiments: {e}&quot;)
            return self._create_error_chart(&quot;Graphique à barres des sentiments&quot;)
    
    def create_keyword_frequency_chart(self, keywords: List[Dict[str, Any]],
                                    title: str = &quot;Mots-clés principaux par fréquence&quot;,
                                    top_n: int = 20) -> plt.Figure:
        &quot;&quot;&quot;Crée un graphique à barres horizontales pour les mots-clés&quot;&quot;&quot;
        try:
            if not keywords:
                return self._create_empty_chart(&quot;Aucun mot-clé disponible&quot;)
            
            fig, ax = plt.subplots(figsize=(10, 8))
            
            # Trier les mots-clés par fréquence
            sorted_keywords = sorted(keywords, key=lambda x: x['frequency'], reverse=True)[:top_n]
            
            # Préparer les données
            keyword_names = [kw['keyword'] for kw in reversed(sorted_keywords)]
            frequencies = [kw['frequency'] for kw in reversed(sorted_keywords)]
            
            # Créer un graphique à barres horizontales
            bars = ax.barh(keyword_names, frequencies, color=self.colors[0], alpha=0.8)
            
            # Personnaliser
            ax.set_title(title, fontsize=14, fontweight='bold')
            ax.set_xlabel('Fréquence')
            
            # Ajouter des étiquettes de valeur
            for bar, freq in zip(bars, frequencies):
                width = bar.get_width()
                ax.text(
                    width + max(frequencies)*0.01, bar.get_y() + bar.get_height()/2.,
                    f'{freq}', ha='left', va='center', fontweight='bold'
                )
            
            # Améliorer la mise en page
            plt.tight_layout()
            
            return fig
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la création du graphique de fréquence des mots-clés: {e}&quot;)
            return self._create_error_chart(&quot;Graphique de fréquence des mots-clés&quot;)
    
    def create_wordcloud(self, keywords: List[Dict[str, Any]], title: str = &quot;Nuage de mots-clés&quot;) -> plt.Figure:
        &quot;&quot;&quot;Crée un nuage de mots à partir des mots-clés&quot;&quot;&quot;
        try:
            if not keywords:
                return self._create_empty_chart(&quot;Aucun mot-clé disponible pour le nuage de mots&quot;)
            
            # Créer un dictionnaire de fréquences pour le nuage de mots
            word_freq = {kw['keyword']: kw['frequency'] for kw in keywords}
            
            # Générer le nuage de mots
            wordcloud = WordCloud(
                width=800, 
                height=400, 
                background_color='white',
                max_words=100,
                colormap='viridis',
                contour_width=1,
                contour_color='steelblue'
            ).generate_from_frequencies(word_freq)
            
            # Afficher le nuage de mots
            fig, ax = plt.subplots(figsize=(12, 8))
            ax.imshow(wordcloud, interpolation='bilinear')
            ax.set_title(title, fontsize=16, fontweight='bold', pad=20)
            ax.axis('off')
            
            plt.tight_layout()
            
            return fig
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la création du nuage de mots: {e}&quot;)
            return self._create_error_chart(&quot;Nuage de mots&quot;)
    
    def _create_error_chart(self, title: str) -> plt.Figure:
        &quot;&quot;&quot;Crée un graphique d'erreur lorsque les données ne sont pas disponibles&quot;&quot;&quot;
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.text(0.5, 0.5, 'Erreur lors de la création du graphique\nDonnées non disponibles', 
               ha='center', va='center', transform=ax.transAxes,
               fontsize=14, color='red')
        ax.set_title(f'{title} - Erreur', fontsize=14, fontweight='bold')
        ax.axis('off')
        return fig
    
    def _create_empty_chart(self, message: str) -> plt.Figure:
        &quot;&quot;&quot;Crée un graphique vide avec un message&quot;&quot;&quot;
        fig, ax = plt.subplots(figsize=(8, 6))
        ax.text(0.5, 0.5, message, ha='center', va='center', transform=ax.transAxes,
               fontsize=14, color='gray')
        ax.axis('off')
        return fig

## 8. Classe principale d'analyse

In [ ]:
# Classe principale d'analyse
class SocialMediaAnalyzer:
    &quot;&quot;&quot;Orchestrateur principal pour l'analyse de sentiment des médias sociaux&quot;&quot;&quot;
    
    def __init__(self):
        self.extractor = None
        self.sentiment_analyzer = None
        self.keyword_extractor = None
        self.text_preprocessor = TextPreprocessor()
        self.charts_generator = ChartsGenerator()
        self.validator = DataValidator()
        
        logger.info(&quot;SocialMediaAnalyzer initialisé&quot;)
    
    def analyze(self, service: str, source: str, days: int = 30, max_posts: int = 500,
               language: str = 'auto', sentiment_model: str = 'auto',
               keyword_method: str = 'combined', progress_callback: Optional[Callable] = None) -> Dict[str, Any]:
        &quot;&quot;&quot;
        Effectue une analyse complète de sentiment des médias sociaux
        
        Args:
            service: Nom du service/marque à analyser
            source: Source de médias sociaux (twitter, facebook, google_reviews)
            days: Nombre de jours à analyser
            max_posts: Nombre maximum de publications à extraire
            language: Langue pour l'analyse
            sentiment_model: Modèle d'analyse de sentiment à utiliser
            keyword_method: Méthode d'extraction de mots-clés
            progress_callback: Fonction de rappel pour les mises à jour de progression
        
        Returns:
            Dictionnaire complet des résultats d'analyse
        &quot;&quot;&quot;
        try:
            start_time = time.time()
            
            if progress_callback:
                progress_callback(f&quot;Démarrage de l'analyse pour '{service}' depuis {source}&quot;)
            
            logger.info(f&quot;Démarrage de l'analyse: service={service}, source={source}, days={days}, max_posts={max_posts}&quot;)
            
            # Étape 1: Extraction des données
            if progress_callback:
                progress_callback(&quot;Extraction des données depuis les médias sociaux...&quot;)
            
            raw_data = self._extract_data(service, source, days, max_posts)
            
            if not raw_data:
                logger.error(&quot;Aucune donnée extraite&quot;)
                return {'error': 'Aucune donnée extraite', 'success': False}
            
            if progress_callback:
                progress_callback(f&quot;Extrait {len(raw_data)} publications&quot;)
            
            # Étape 2: Prétraitement du texte
            if progress_callback:
                progress_callback(&quot;Prétraitement des données textuelles...&quot;)
            
            processed_data = self._preprocess_data(raw_data, language)
            
            # Étape 3: Analyse de sentiment
            if progress_callback:
                progress_callback(&quot;Analyse du sentiment...&quot;)
            
            sentiment_results = self._analyze_sentiment(processed_data, sentiment_model, language)
            
            # Étape 4: Extraction de mots-clés
            if progress_callback:
                progress_callback(&quot;Extraction des mots-clés...&quot;)
            
            keywords = self._extract_keywords(processed_data, keyword_method)
            
            # Étape 5: Analyse temporelle (si les données de date sont disponibles)
            temporal_analysis = None
            if any('created_at' in item for item in processed_data):
                if progress_callback:
                    progress_callback(&quot;Analyse des tendances temporelles...&quot;)
                temporal_analysis = self._analyze_temporal_trends(sentiment_results, processed_data)
            
            # Étape 6: Générer des statistiques de résumé
            if progress_callback:
                progress_callback(&quot;Génération des statistiques de résumé...&quot;)
            
            summary_stats = self._generate_summary_statistics(
                sentiment_results, keywords, raw_data, processed_data
            )
            
            # Étape 7: Compiler les résultats
            if progress_callback:
                progress_callback(&quot;Compilation des résultats finaux...&quot;)
            
            results = self._compile_results(
                service, source, days, max_posts, raw_data, processed_data,
                sentiment_results, keywords, temporal_analysis, summary_stats
            )
            
            execution_time = time.time() - start_time
            
            results['metadata']['execution_time'] = execution_time
            
            logger.info(f&quot;Analyse terminée avec succès en {execution_time:.2f} secondes&quot;)
            
            if progress_callback:
                progress_callback(f&quot;Analyse terminée en {execution_time:.2f} secondes&quot;)
            
            return results
            
        except Exception as e:
            logger.error(f&quot;L'analyse a échoué: {e}&quot;)
            return {'error': str(e), 'success': False}
    
    def _extract_data(self, service: str, source: str, days: int, max_posts: int) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les données de la source de médias sociaux&quot;&quot;&quot;
        try:
            # Initialiser l'extracteur approprié
            if source.lower() == 'twitter':
                self.extractor = TwitterExtractor(service, max_posts)
            else:
                raise ValueError(f&quot;Source non prise en charge: {source}&quot;)
            
            # Extraire les données
            raw_data = self.extractor.extract_posts(days=days)
            
            logger.info(f&quot;Extrait {len(raw_data)} publications depuis {source}&quot;)
            return raw_data
            
        except ExtractionError as e:
            logger.error(f&quot;L'extraction des données a échoué: {e}&quot;)
            raise
        except Exception as e:
            logger.error(f&quot;Erreur inattendue lors de l'extraction des données: {e}&quot;)
            raise
    
    def _preprocess_data(self, raw_data: List[Dict[str, Any]], language: str) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Prétraite les données extraites&quot;&quot;&quot;
        processed_data = []
        
        for item in tqdm(raw_data, desc=&quot;Prétraitement des données&quot;):
            try:
                # Extraire le contenu textuel
                text = item.get('text', item.get('message', item.get('content', '')))
                
                if not text or not self.validator.validate_text_content(text):
                    continue
                
                # Prétraiter le texte
                preprocessing_result = self.text_preprocessor.preprocess_text(text, language)
                
                # Créer un élément prétraité
                processed_item = item.copy()
                processed_item.update({
                    'cleaned_text': preprocessing_result['cleaned'],
                    'tokens': preprocessing_result['tokens'],
                    'language': preprocessing_result['language'],
                    'preprocessing_steps': preprocessing_result['preprocessing_steps']
                })
                
                processed_data.append(processed_item)
                
            except Exception as e:
                logger.warning(f&quot;Erreur lors du prétraitement de l'élément {item.get('id', 'inconnu')}: {e}&quot;)
                continue
        
        logger.info(f&quot;Prétraité {len(processed_data)} éléments&quot;)
        return processed_data
    
    def _analyze_sentiment(self, processed_data: List[Dict[str, Any]], 
                         sentiment_model: str, language: str) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Analyse le sentiment des données prétraitées&quot;&quot;&quot;
        self.sentiment_analyzer = SentimentAnalyzer(sentiment_model, language)
        
        sentiment_results = []
        texts = [item['cleaned_text'] for item in processed_data]
        
        # Analyser le sentiment pour chaque texte
        for i, (item, text) in enumerate(tqdm(zip(processed_data, texts), desc=&quot;Analyse de sentiment&quot;, total=len(texts))):
            try:
                sentiment_result = self.sentiment_analyzer.analyze_sentiment(
                    text, item.get('language', language)
                )
                
                # Ajouter des métadonnées
                sentiment_result.update({
                    'id': item.get('id'),
                    'original_text': item.get('text', ''),
                    'date': item.get('created_at'),
                    'source': item.get('source'),
                    'service': item.get('service')
                })
                
                sentiment_results.append(sentiment_result)
                
            except Exception as e:
                logger.warning(f&quot;Erreur lors de l'analyse du sentiment pour l'élément {item.get('id', i)}: {e}&quot;)
                continue
        
        logger.info(f&quot;Sentiment analysé pour {len(sentiment_results)} éléments&quot;)
        return sentiment_results
    
    def _extract_keywords(self, processed_data: List[Dict[str, Any]], 
                        keyword_method: str) -> List[Dict[str, Any]]:
        &quot;&quot;&quot;Extrait les mots-clés des données prétraitées&quot;&quot;&quot;
        self.keyword_extractor = KeywordExtractor(
            language='auto',  # Utiliser la détection automatique
            max_keywords=50
        )
        
        # Extraire les textes pour l'analyse des mots-clés
        texts = [item['cleaned_text'] for item in processed_data if item.get('cleaned_text')]
        
        if not texts:
            logger.warning(&quot;Aucun texte disponible pour l'extraction de mots-clés&quot;)
            return []
        
        # Extraire les mots-clés
        keywords = self.keyword_extractor.extract_keywords(texts, keyword_method)
        
        # Extraire également les phrases clés
        key_phrases = self.keyword_extractor.extract_key_phrases(texts)
        
        # Combiner les mots-clés et les phrases
        all_keywords = keywords + key_phrases
        
        # Trier par score et limiter
        all_keywords.sort(key=lambda x: x.get('score', 0), reverse=True)
        
        logger.info(f&quot;Extrait {len(all_keywords)} mots-clés/phrases&quot;)
        return all_keywords[:50]  # Limiter aux 50 premiers
    
    def _analyze_temporal_trends(self, sentiment_results: List[Dict[str, Any]], 
                               processed_data: List[Dict[str, Any]]) -> Dict[str, Any]:
        &quot;&quot;&quot;Analyse les tendances temporelles du sentiment&quot;&quot;&quot;
        try:
            # Préparer les données pour l'analyse temporelle
            texts = []
            dates = []
            
            for result, item in zip(sentiment_results, processed_data):
                if item.get('created_at'):
                    texts.append(result.get('original_text', ''))
                    dates.append(item['created_at'])
            
            if not texts or not dates:
                logger.warning(&quot;Données temporelles insuffisantes pour l'analyse des tendances&quot;)
                return {}
            
            # Créer un DataFrame pour l'analyse
            df = pd.DataFrame({
                'date': pd.to_datetime(dates),
                'sentiment': [r['sentiment'] for r in sentiment_results if 'sentiment' in r],
                'polarity': [r['polarity'] for r in sentiment_results if 'polarity' in r]
            })
            
            # Grouper par jour
            df['date'] = df['date'].dt.date
            daily = df.groupby('date').agg({
                'sentiment': lambda x: x.value_counts().to_dict(),
                'polarity': 'mean'
            }).reset_index()
            
            # Convertir en format de résultat
            trends = {
                'daily_sentiment': daily.to_dict('records'),
                'overall_trend': 'stable',  # Simplifié pour cet exemple
                'detailed_results': sentiment_results
            }
            
            return trends
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de l'analyse des tendances temporelles: {e}&quot;)
            return {}
    
    def _generate_summary_statistics(self, sentiment_results: List[Dict[str, Any]],
                                  keywords: List[Dict[str, Any]], 
                                  raw_data: List[Dict[str, Any]],
                                  processed_data: List[Dict[str, Any]]) -> Dict[str, Any]:
        &quot;&quot;&quot;Génère des statistiques de résumé&quot;&quot;&quot;
        try:
            # Statistiques de sentiment
            sentiment_stats = self.sentiment_analyzer.get_sentiment_summary(sentiment_results)
            
            # Statistiques d'extraction
            extraction_stats = self.extractor.get_extraction_stats() if self.extractor else {}
            
            # Statistiques de traitement
            processing_stats = {
                'raw_data_count': len(raw_data),
                'processed_data_count': len(processed_data),
                'processing_success_rate': (len(processed_data) / len(raw_data) * 100) if raw_data else 0
            }
            
            # Statistiques de mots-clés
            keyword_stats = {
                'total_keywords': len(keywords),
                'avg_keyword_score': sum(kw.get('score', 0) for kw in keywords) / len(keywords) if keywords else 0,
                'top_keyword': keywords[0]['keyword'] if keywords else None
            }
            
            return {
                'sentiment_stats': sentiment_stats,
                'extraction_stats': extraction_stats,
                'processing_stats': processing_stats,
                'keyword_stats': keyword_stats
            }
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la génération des statistiques de résumé: {e}&quot;)
            return {}
    
    def _compile_results(self, service: str, source: str, days: int, max_posts: int,
                       raw_data: List[Dict[str, Any]], processed_data: List[Dict[str, Any]],
                       sentiment_results: List[Dict[str, Any]], keywords: List[Dict[str, Any]],
                       temporal_analysis: Optional[Dict[str, Any]], 
                       summary_stats: Dict[str, Any]) -> Dict[str, Any]:
        &quot;&quot;&quot;Compile tous les résultats dans une structure finale&quot;&quot;&quot;
        try:
            results = {
                'metadata': {
                    'service': service,
                    'source': source,
                    'analysis_date': datetime.now().isoformat(),
                    'parameters': {
                        'days': days,
                        'max_posts': max_posts,
                        'language': 'auto',
                        'sentiment_model': 'auto',
                        'keyword_method': 'combined'
                    }
                },
                'raw_data': raw_data,
                'processed_data': processed_data,
                'sentiment_results': sentiment_results,
                'sentiment_summary': summary_stats.get('sentiment_stats', {}),
                'keywords': keywords,
                'temporal_data': temporal_analysis.get('detailed_results', []) if temporal_analysis else [],
                'statistics': summary_stats,
                'success': True
            }
            
            return results
            
        except Exception as e:
            logger.error(f&quot;Erreur lors de la compilation des résultats: {e}&quot;)
            return {'error': str(e), 'success': False}

## 9. Interface utilisateur pour l'analyse

In [ ]:
# Interface utilisateur pour l'analyse
class SentimentAnalysisUI:
    &quot;&quot;&quot;Interface utilisateur pour l'analyse de sentiment&quot;&quot;&quot;
    
    def __init__(self):
        self.analyzer = SocialMediaAnalyzer()
        self.results = None
        self.progress_output = widgets.Output()
        self.charts_output = widgets.Output()
        self.data_output = widgets.Output()
    
    def create_ui(self):
        &quot;&quot;&quot;Crée l'interface utilisateur&quot;&quot;&quot;
        # Paramètres d'entrée
        self.service_input = widgets.Text(
            value='Apple',
            description='Service/Marque:',
            style={'description_width': 'initial'}
        )
        
        self.source_dropdown = widgets.Dropdown(
            options=['twitter'],
            value='twitter',
            description='Source:',
            style={'description_width': 'initial'}
        )
        
        self.days_slider = widgets.IntSlider(
            value=7,
            min=1,
            max=30,
            step=1,
            description='Jours:',
            style={'description_width': 'initial'}
        )
        
        self.max_posts_slider = widgets.IntSlider(
            value=100,
            min=10,
            max=500,
            step=10,
            description='Max posts:',
            style={'description_width': 'initial'}
        )
        
        self.language_dropdown = widgets.Dropdown(
            options=['auto', 'english', 'french'],
            value='auto',
            description='Langue:',
            style={'description_width': 'initial'}
        )
        
        self.model_dropdown = widgets.Dropdown(
            options=['auto', 'textblob', 'transformers'],
            value='auto',
            description='Modèle:',
            style={'description_width': 'initial'}
        )
        
        # Bouton d'analyse
        self.analyze_button = widgets.Button(
            description='Lancer l\'analyse',
            button_style='primary',
            icon='play'
        )
        self.analyze_button.on_click(self._on_analyze_click)
        
        # Onglets pour les résultats
        tabs = widgets.Tab([
            self.progress_output,
            self.charts_output,
            self.data_output
        ])
        tabs.set_title(0, 'Progression')
        tabs.set_title(1, 'Graphiques')
        tabs.set_title(2, 'Données')
        
        # Mise en page
        input_widgets = widgets.VBox([
            widgets.HBox([self.service_input, self.source_dropdown]),
            widgets.HBox([self.days_slider, self.max_posts_slider]),
            widgets.HBox([self.language_dropdown, self.model_dropdown]),
            self.analyze_button
        ])
        
        # Afficher l'interface
        display(widgets.VBox([input_widgets, tabs]))
    
    def _on_analyze_click(self, b):
        &quot;&quot;&quot;Gestionnaire d'événements pour le bouton d'analyse&quot;&quot;&quot;
        # Désactiver le bouton pendant l'analyse
        self.analyze_button.disabled = True
        
        # Effacer les sorties précédentes
        self.progress_output.clear_output()
        self.charts_output.clear_output()
        self.data_output.clear_output()
        
        # Obtenir les paramètres
        service = self.service_input.value
        source = self.source_dropdown.value
        days = self.days_slider.value
        max_posts = self.max_posts_slider.value
        language = self.language_dropdown.value
        model = self.model_dropdown.value
        
        # Fonction de rappel pour la progression
        def progress_callback(message):
            with self.progress_output:
                print(f&quot;{datetime.now().strftime('%H:%M:%S')} - {message}&quot;)
        
        # Lancer l'analyse dans un thread séparé
        import threading
        
        def run_analysis():
            try:
                with self.progress_output:
                    print(f&quot;Démarrage de l'analyse pour '{service}' depuis {source}...&quot;)
                
                # Exécuter l'analyse
                self.results = self.analyzer.analyze(
                    service=service,
                    source=source,
                    days=days,
                    max_posts=max_posts,
                    language=language,
                    sentiment_model=model,
                    progress_callback=progress_callback
                )
                
                # Afficher les résultats
                self._display_results()
                
            except Exception as e:
                with self.progress_output:
                    print(f&quot;Erreur lors de l'analyse: {e}&quot;)
            finally:
                # Réactiver le bouton
                self.analyze_button.disabled = False
        
        # Démarrer l'analyse dans un thread
        thread = threading.Thread(target=run_analysis)
        thread.start()
    
    def _display_results(self):
        &quot;&quot;&quot;Affiche les résultats de l'analyse&quot;&quot;&quot;
        if not self.results or not self.results.get('success', False):
            with self.progress_output:
                print(&quot;L'analyse a échoué ou n'a pas produit de résultats.&quot;)
                if 'error' in self.results:
                    print(f&quot;Erreur: {self.results['error']}&quot;)
            return
        
        # Afficher les graphiques
        with self.charts_output:
            clear_output(wait=True)
            
            # Créer les graphiques
            print(&quot;## Graphiques d'analyse de sentiment&quot;)
            
            # Graphique circulaire des sentiments
            if 'sentiment_summary' in self.results:
                fig_pie = self.analyzer.charts_generator.create_sentiment_pie_chart(
                    self.results['sentiment_summary'],
                    title=f&quot;Distribution des sentiments pour {self.results['metadata']['service']}&quot;
                )
                plt.show(fig_pie)
            
            # Graphique à barres des sentiments
            if 'sentiment_summary' in self.results:
                fig_bar = self.analyzer.charts_generator.create_sentiment_bar_chart(
                    self.results['sentiment_summary'],
                    title=f&quot;Analyse de sentiment pour {self.results['metadata']['service']}&quot;
                )
                plt.show(fig_bar)
            
            # Nuage de mots-clés
            if 'keywords' in self.results and self.results['keywords']:
                fig_wordcloud = self.analyzer.charts_generator.create_wordcloud(
                    self.results['keywords'],
                    title=f&quot;Nuage de mots-clés pour {self.results['metadata']['service']}&quot;
                )
                plt.show(fig_wordcloud)
            
            # Graphique de fréquence des mots-clés
            if 'keywords' in self.results and self.results['keywords']:
                fig_keywords = self.analyzer.charts_generator.create_keyword_frequency_chart(
                    self.results['keywords'],
                    title=f&quot;Mots-clés principaux pour {self.results['metadata']['service']}&quot;,
                    top_n=15
                )
                plt.show(fig_keywords)
        
        # Afficher les données
        with self.data_output:
            clear_output(wait=True)
            
            print(&quot;## Résumé de l'analyse&quot;)
            print(f&quot;Service analysé: {self.results['metadata']['service']}&quot;)
            print(f&quot;Source: {self.results['metadata']['source']}&quot;)
            print(f&quot;Date d'analyse: {self.results['metadata']['analysis_date']}&quot;)
            print(f&quot;Temps d'exécution: {self.results['metadata']['execution_time']:.2f} secondes&quot;)
            print(&quot;\n&quot;)
            
            print(&quot;### Statistiques de sentiment&quot;)
            sentiment_stats = self.results['sentiment_summary']
            print(f&quot;Total des publications: {sentiment_stats['total']}&quot;)
            print(f&quot;Positif: {sentiment_stats['positive']} ({sentiment_stats['percentages']['positive']:.1f}%)&quot;)
            print(f&quot;Négatif: {sentiment_stats['negative']} ({sentiment_stats['percentages']['negative']:.1f}%)&quot;)
            print(f&quot;Neutre: {sentiment_stats['neutral']} ({sentiment_stats['percentages']['neutral']:.1f}%)&quot;)
            print(f&quot;Polarité moyenne: {sentiment_stats['average_polarity']:.3f}&quot;)
            print(&quot;\n&quot;)
            
            print(&quot;### Mots-clés principaux&quot;)
            if 'keywords' in self.results and self.results['keywords']:
                top_keywords = self.results['keywords'][:10]
                for i, kw in enumerate(top_keywords, 1):
                    print(f&quot;{i}. {kw['keyword']} (score: {kw['score']:.3f}, fréquence: {kw.get('frequency', 'N/A')})&quot;)
            else:
                print(&quot;Aucun mot-clé extrait&quot;)
            print(&quot;\n&quot;)
            
            print(&quot;### Exemples de publications&quot;)
            if 'sentiment_results' in self.results and self.results['sentiment_results']:
                # Afficher quelques exemples de chaque catégorie
                sentiment_results = self.results['sentiment_results']
                
                # Positif
                positive = [r for r in sentiment_results if r['sentiment'] == 'positive'][:3]
                if positive:
                    print(&quot;\n**Publications positives:**&quot;)
                    for i, p in enumerate(positive, 1):
                        print(f&quot;{i}. \&quot;{p['original_text'][:100]}...\&quot; (polarité: {p['polarity']:.2f})&quot;)
                
                # Négatif
                negative = [r for r in sentiment_results if r['sentiment'] == 'negative'][:3]
                if negative:
                    print(&quot;\n**Publications négatives:**&quot;)
                    for i, n in enumerate(negative, 1):
                        print(f&quot;{i}. \&quot;{n['original_text'][:100]}...\&quot; (polarité: {n['polarity']:.2f})&quot;)
                
                # Neutre
                neutral = [r for r in sentiment_results if r['sentiment'] == 'neutral'][:3]
                if neutral:
                    print(&quot;\n**Publications neutres:**&quot;)
                    for i, n in enumerate(neutral, 1):
                        print(f&quot;{i}. \&quot;{n['original_text'][:100]}...\&quot; (polarité: {n['polarity']:.2f})&quot;)
            else:
                print(&quot;Aucun résultat d'analyse de sentiment disponible&quot;)

## 10. Lancement de l'interface utilisateur

In [ ]:
# Créer et afficher l'interface utilisateur
ui = SentimentAnalysisUI()
ui.create_ui()

## 11. Exemple d'analyse manuelle (alternative à l'interface utilisateur)

In [ ]:
# Exemple d'utilisation manuelle (sans interface utilisateur)
def run_manual_analysis():
    &quot;&quot;&quot;Exécute une analyse manuelle sans l'interface utilisateur&quot;&quot;&quot;
    # Paramètres d'analyse
    service = &quot;Apple&quot;  # Remplacez par le service/marque que vous souhaitez analyser
    source = &quot;twitter&quot;
    days = 7
    max_posts = 100
    
    # Fonction de rappel pour la progression
    def progress_callback(message):
        print(f&quot;{datetime.now().strftime('%H:%M:%S')} - {message}&quot;)
    
    # Créer l'analyseur et exécuter l'analyse
    analyzer = SocialMediaAnalyzer()
    results = analyzer.analyze(
        service=service,
        source=source,
        days=days,
        max_posts=max_posts,
        progress_callback=progress_callback
    )
    
    # Vérifier si l'analyse a réussi
    if not results or not results.get('success', False):
        print(&quot;L'analyse a échoué ou n'a pas produit de résultats.&quot;)
        if 'error' in results:
            print(f&quot;Erreur: {results['error']}&quot;)
        return
    
    # Afficher les résultats
    print(&quot;\n## Résumé de l'analyse&quot;)
    print(f&quot;Service analysé: {results['metadata']['service']}&quot;)
    print(f&quot;Source: {results['metadata']['source']}&quot;)
    print(f&quot;Date d'analyse: {results['metadata']['analysis_date']}&quot;)
    print(f&quot;Temps d'exécution: {results['metadata']['execution_time']:.2f} secondes&quot;)
    
    # Afficher les statistiques de sentiment
    sentiment_stats = results['sentiment_summary']
    print(&quot;\n### Statistiques de sentiment&quot;)
    print(f&quot;Total des publications: {sentiment_stats['total']}&quot;)
    print(f&quot;Positif: {sentiment_stats['positive']} ({sentiment_stats['percentages']['positive']:.1f}%)&quot;)
    print(f&quot;Négatif: {sentiment_stats['negative']} ({sentiment_stats['percentages']['negative']:.1f}%)&quot;)
    print(f&quot;Neutre: {sentiment_stats['neutral']} ({sentiment_stats['percentages']['neutral']:.1f}%)&quot;)
    
    # Créer et afficher les graphiques
    charts_generator = ChartsGenerator()
    
    # Graphique circulaire des sentiments
    fig_pie = charts_generator.create_sentiment_pie_chart(
        results['sentiment_summary'],
        title=f&quot;Distribution des sentiments pour {results['metadata']['service']}&quot;
    )
    plt.show(fig_pie)
    
    # Nuage de mots-clés
    if 'keywords' in results and results['keywords']:
        fig_wordcloud = charts_generator.create_wordcloud(
            results['keywords'],
            title=f&quot;Nuage de mots-clés pour {results['metadata']['service']}&quot;
        )
        plt.show(fig_wordcloud)
    
    return results

# Décommentez la ligne suivante pour exécuter l'analyse manuelle
# results = run_manual_analysis()

## 12. Documentation et limitations

### Limitations

1. **Extraction de données**:
   - Cette version utilise uniquement snscrape pour Twitter, qui peut être limité par les politiques de Twitter
   - Facebook et Google Reviews ne sont pas implémentés dans cette version Colab

2. **Analyse de sentiment**:
   - TextBlob offre une analyse de sentiment basique qui peut manquer de précision pour certains contextes
   - Le modèle Transformers nécessite un téléchargement initial qui peut prendre du temps

3. **Performance**:
   - Les ressources de Colab peuvent limiter la taille des analyses
   - L'extraction d'un grand nombre de posts peut être lente

### Conseils d'utilisation

1. Commencez avec un petit nombre de publications (50-100) pour tester
2. Pour des analyses plus rapides, utilisez TextBlob au lieu de Transformers
3. Sauvegardez les résultats importants car les sessions Colab peuvent expirer
4. Pour des analyses plus précises en français, utilisez le modèle Transformers

### Extensions possibles

1. Ajouter d'autres sources de médias sociaux
2. Implémenter une analyse de sentiment plus avancée
3. Ajouter des fonctionnalités d'exportation des résultats
4. Intégrer une analyse d'émotions plus détaillée
